In [1]:
!pip install fastf1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 13.1 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [6]:
import fastf1
import matplotlib.pyplot as plt
import pandas as pd
import os

# Enable Cache

if not os.path.exists('cache'):
  os.makedirs('cache')
fastf1.Cache.enable_cache('cache')

# Load session
session = fastf1.get_session(2025, 'Mexico City', 'R')
session.load()

# Drivers involved in both battles
drivers = ['VER', 'LEC', 'PIA', 'BEA']
laps = session.laps.pick_drivers(drivers).pick_quicklaps()


# Define lap window (pre-VSC)
LAP_START = 60
LAP_END = 70  # VSC begins at start of lap 70
laps_window = laps[(laps['LapNumber'] >= LAP_START) & (laps['LapNumber'] < LAP_END)].copy()


# Helper function to compute per-lap pace and deltas
def compute_pace_trend(laps_df, leader, chaser, lap_no_for_gap=60):
    """Compute per-lap pace trend and projected gap between leader and chaser."""

    # Filter data for each driver
    lead = laps_df[laps_df['Driver'] == leader][['LapNumber', 'LapTime']]
    chase = laps_df[laps_df['Driver'] == chaser][['LapNumber', 'LapTime']]

    # Merge and compute lap-time differences
    df = lead.merge(chase, on='LapNumber', suffixes=('_lead', '_chaser'))
    df['LeadTime'] = df['LapTime_lead'].dt.total_seconds()
    df['ChaserTime'] = df['LapTime_chaser'].dt.total_seconds()
    df['GainPerLap'] = df['LeadTime'] - df['ChaserTime']  # positive = chaser faster
    df['CumulativeGain'] = df['GainPerLap'].cumsum()

    # Calculate actual gap at lap_no_for_gap
    leader_lap = laps[(laps['Driver'] == leader) & (laps['LapNumber'] == lap_no_for_gap)]
    chaser_lap = laps[(laps['Driver'] == chaser) & (laps['LapNumber'] == lap_no_for_gap)]
    gap_init = (
        chaser_lap['LapStartTime'].iloc[0] - leader_lap['LapStartTime'].iloc[0]
    ).total_seconds()

    # Project gap evolution
    df['ProjectedGap'] = gap_init - df['CumulativeGain']

    return df, gap_init


# Compute for both battles
pairs = {
    'P2 Battle (LEC vs VER)': ('LEC', 'VER'),
    'P4 Battle (BEA vs PIA)': ('BEA', 'PIA')
}

results = {}

for title, (leader, chaser) in pairs.items():
    df, gap_init = compute_pace_trend(laps_window, leader, chaser)
    results[title] = df
    print(f"\n=== {title} ===")
    print(f"Initial gap at lap 60: {gap_init:.2f}s")
    avg_gain = df['GainPerLap'].mean()
    print(f"Average pace gain per lap by {chaser}: {avg_gain:.3f}s")
    print(f"Projected gap at lap 71 (2 laps more): {df['ProjectedGap'].iloc[-1] - avg_gain*2:.3f}s")

core           INFO 	Loading data for Mexico City Grand Prix - Race [v3.6.1]
INFO:fastf1.fastf1.core:Loading data for Mexico City Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
INFO:fastf1.fastf1.req:Using cached data for session_info
req            INFO 	Using cached data for driver_info
INFO:fastf1.fastf1.req:Using cached data for driver_info
DEBUG:fastf1.ergast:Failed to parse timestamp '-1:55:35.558' in Ergastresponse.
req            INFO 	Using cached data for session_status_data
INFO:fastf1.fastf1.req:Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
INFO:fastf1.fastf1.req:Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
INFO:fastf1.fastf1.req:Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
INFO:fastf1.fastf1.req:Using cached data for _extended_timing_data
req            INFO 	Using cached data for ti


=== P2 Battle (LEC vs VER) ===
Initial gap at lap 60: 5.60s
Average pace gain per lap by VER: 0.498s
Projected gap at lap 71 (2 laps more): -0.384s

=== P4 Battle (BEA vs PIA) ===
Initial gap at lap 60: 2.18s
Average pace gain per lap by PIA: 0.104s
Projected gap at lap 71 (2 laps more): 0.930s
